# Analysis of Orbital Evolution in Stationary & Axisymmetric Spacetimes.

Consider an Extreme Mass Ratio Inspiral (EMRI) system. Let the masses of the secondary and the primary be $\mu=2M_\odot$ and $M=10^6\times \mu.$

Since the mass of the secondary is negligible compared to the mass of the primary, its motion in the gravitational field generated by the primary at the zeroth order can be approximated by geodesic motion.

In [1]:
from functools import reduce
from joblib import Parallel, delayed, cpu_count
from matplotlib_inline.backend_inline import set_matplotlib_formats
import matplotlib.pyplot as plt
from numba import njit
import numpy as np
from os import mkdir
from pandas import DataFrame, read_csv
from scipy.fft import fft, fftfreq
from scipy.integrate import solve_ivp
from sympy import symbols, diff, lambdify, sin, cos, sqrt
from warnings import simplefilter

# Disable furure warnings.
simplefilter(action='ignore', category=FutureWarning)

set_matplotlib_formats('svg')
%matplotlib widget